# Test 5: Analyze Truth Jet with Scheme 1
Test my analysis function with scheme 1 of truth jet by using SVJ with CKKW-L and without decay data.

## 1. Import Packages

In [1]:
# The Python Standard Library
import os
import sys
import time
import datetime
import glob
import multiprocessing as mp

# The Third-Party Library
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import prettytable
import uproot
import pyjet
import importlib

# My Packages
import myhep.particle_information_v2 as mypInfo_v2
import myhep.analytical_function_v2 as myaFun_v2
import myhep.analysis_v3 as myAnal_v3
# import myhep.particleinfo_v1 as mypiv1
# import myhep.particlefun_v1 as myafv1

# increase figure showing resolution
%config InlineBackend.figure_format = 'retina'

## 2. Import .root File and Load the Data via class

In [2]:
INPUT_FILE = '/youwei_u3/svj_data_master/scheme_1/root/ckkwl_wo.root'

DATA = uproot.open(INPUT_FILE)['Delphes;1']
GP = mypInfo_v2.classGenParticle(DATA)
Jet = mypInfo_v2.classJet(DATA)
Event = mypInfo_v2.classEvent(DATA)

### 2-1. Check the number of events for each branch
Skip this step, since I already checked many times.

### 2-2. Define mass quantities of 2 objects

In [3]:
# 2-1. Mass quantities of two objects
# A. Invariant mass M
def M(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    return np.sqrt((e1+e2)**2 - (px1+px2)**2 - (py1+py2)**2 - (pz1+pz2)**2)


# B. Transverse mass MT
def MT(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    ET1, ET2 = np.sqrt(m1**2 + pt1**2), np.sqrt(m2**2 + pt2**2)
    return np.sqrt((ET1+ET2)**2 - (px1+px2)**2 - (py1+py2)**2)


# C. Transverse mass mT
# * mT is invariant under Lorentz boost along the z direction.
def mT(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    return np.sqrt((e1+e2)**2 - (pz1+pz2)**2)


# D. Transverse energy ET
def ET(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    m12 = np.sqrt((e1+e2)**2 - (px1+px2)**2 - (py1+py2)**2 - (pz1+pz2)**2)
    return np.sqrt(m12**2 + (px1+px2)**2 + (py1+py2)**2)

### 2-3. Define mass quantities of 3 objects

In [4]:
# 2-2. Mass quantities of three objects
# A. Invariant mass M
def M_123(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2, m3, pt3, eta3, phi3):
    """Invariant mass M of 3 objects, M is defined by M^2 = E^2 - p^2.
    Parameters
    ----------
    m1, m2, m3 : float
        Mass of object i.
    pt1, pt2, pt3 : float
        Transverse momentum of object i.
    eta1, eta2, eta3 : float
        Pseudorapidity of object i.
    phi1, phi2, phi3 : float
        Azimuthal angle of object i.
    Returns
    -------
    float
        Invariant mass M.
    """
    # object 1
    px1, py1 = pt1 * np.cos(phi1), pt1 * np.sin(phi1)
    pz1 = np.sqrt(m1**2 + pt1**2) * np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    # object 2
    px2, py2 = pt2 * np.cos(phi2), pt2 * np.sin(phi2)
    pz2 = np.sqrt(m2**2 + pt2**2) * np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    # object 3
    px3, py3 = pt3 * np.cos(phi3), pt3 * np.sin(phi3)
    pz3 = np.sqrt(m3**2 + pt3**2) * np.sinh(eta3)
    e3 = np.sqrt(m3**2 + px3**2 + py3**2 + pz3**2)
    return np.sqrt((e1 + e2 + e3)**2
                   - (px1 + px2 + px3)**2
                   - (py1 + py2 + py3)**2
                   - (pz1 + pz2 + pz3)**2)


# B. Transverse mass MT
def MT_123(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2, m3, pt3, eta3, phi3):
    """Transverse mass MT of 3 objects, MT is defined by
    M_T^2 = (E_T(1) + E_T(2))^2 - (p_T(1) + p_T(2))^2.
    Parameters
    ----------
    m1, m2, m3 : float
        Mass of object i.
    pt1, pt2, pt3 : float
        Transverse momentum of object i.
    eta1, eta2, eta2 : float
        Pseudorapidity of object i.
    phi1, phi2, phi3 : float
        Azimuthal angle of object i.
    Returns
    -------
    float
        Transverse mass MT.
    """
    # object 1
    px1, py1 = pt1 * np.cos(phi1), pt1 * np.sin(phi1)
    pz1 = np.sqrt(m1**2 + pt1**2) * np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    et1 = np.sqrt(m1**2 + pt1**2)
    # object 2
    px2, py2 = pt2 * np.cos(phi2), pt2 * np.sin(phi2)
    pz2 = np.sqrt(m2**2 + pt2**2) * np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    et2 = np.sqrt(m2**2 + pt2**2)
    # object 3
    px3, py3 = pt3 * np.cos(phi3), pt3 * np.sin(phi3)
    pz3 = np.sqrt(m3**2 + pt3**2) * np.sinh(eta3)
    e3 = np.sqrt(m3**2 + px3**2 + py3**2 + pz3**2)
    et3 = np.sqrt(m3**2 + pt3**2)
    return np.sqrt((et1 + et2 + et3)**2
                   - (px1 + px2 + px3)**2
                   - (py1 + py2 + py3)**2)


# C. Transverse mass mT with definition 1
# * mT is invariant under Lorentz boost along the z direction.
def mT_123_def_1(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2, m3, pt3, eta3, phi3):
    """Transverse mass mT of 3 objects, mT is defined by m_T^2 = E^2 - p_z^2.
    Parameters
    ----------
    m1, m2, m3 : float
        Mass of object i.
    pt1, pt2, pt3 : float
        Transverse mass of object i.
    eta1, eta2, eta3 : float
        Pseudorapidity of object i.
    phi1, phi2, phi3 : float
        Azimuthal angle of object i.
    Returns
    -------
    float
        Transverse mass mT.
    """
    # object 1
    px1, py1 = pt1 * np.cos(phi1), pt1 * np.sin(phi1)
    pz1 = np.sqrt(m1**2 + pt1**2) * np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    # object 2
    px2, py2 = pt2 * np.cos(phi2), pt2 * np.sin(phi2)
    pz2 = np.sqrt(m2**2 + pt2**2) * np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    # object 3
    px3, py3 = pt3 * np.cos(phi3), pt3 * np.sin(phi3)
    pz3 = np.sqrt(m3**2 + pt3**2) * np.sinh(eta3)
    e3 = np.sqrt(m3**2 + px3**2 + py3**2 + pz3**2)
    return np.sqrt((e1 + e2 + e3)**2 - (pz1 + pz2 + pz3)**2)


# D. Transverse mass mT with definition 2 or Transverse energy ET
# * mT is invariant under Lorentz boost along the z direction.
def mT_123_def_2(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2, m3, pt3, eta3, phi3):
    """Transverse mass mT of 3 objects, mT is defined by m_T^2 = m^2 + p_T^2.
    Parameters
    ----------
    m1, m2, m3 : float
        Mass of object i.
    pt1, pt2, pt3 : float
        Transverse momentum of object i.
    eta1, eta2, eta3 : float
        Pseudorapidity of object i.
    phi1, phi2, phi3 : float
        Azimuthal angle of object i.
    Returns
    -------
    float
        Transverse mass mT.
    """
    # object 1
    px1, py1 = pt1 * np.cos(phi1), pt1 * np.sin(phi1)
    pz1 = np.sqrt(m1**2 + pt1**2) * np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    # object 2
    px2, py2 = pt2 * np.cos(phi2), pt2 * np.sin(phi2)
    pz2 = np.sqrt(m2**2 + pt2**2) * np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    # object 3
    px3, py3 = pt3 * np.cos(phi3), pt3 * np.sin(phi3)
    pz3 = np.sqrt(m3**2 + pt3**2) * np.sinh(eta3)
    e3 = np.sqrt(m3**2 + px3**2 + py3**2 + pz3**2)
    # invariant mass of 3 objects
    m123 = np.sqrt((e1 + e2 + e3)**2 - (px1 + px2 + px3)**2
                   - (py1 + py2 + py3)**2 - (pz1 + pz2 + pz3)**2)
    return np.sqrt(m123**2 + (px1 + px2 + px3)**2 + (py1 + py2 + py3)**2)

## 3. Analyze the Dark Quark Pair in the Parton and Truth Levels
Skip this step

## 4. Jet Clustering

### 4-1. Select stable final state particles without/with filtering out dark sector

In [5]:
SFSP_v3, SFSP_filterDM_v3 = myAnal_v3.selectStableFinalStateParticle(
    GP, filter=[51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213])

The PID of dark matter are [51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213].
19373 events are stable final state.
19373 events are stable final state without DM.


### 4-2. Let's do the jet clustering!!

In [6]:
R, jetClusteringAlgorithm, pTmin_pyjet = 0.4, -1, 0

PseudoJet_v3 = myAnal_v3.jetClustering_v1(SFSP_v3, R=R,
                                       p=jetClusteringAlgorithm,
                                       pTmin=pTmin_pyjet)
PseudoJet_filterDM_v3 = myAnal_v3.jetClustering_v1(SFSP_filterDM_v3, R=R,
                                                p=jetClusteringAlgorithm,
                                                pTmin=pTmin_pyjet)
print('Done new version')

Done new version


## 5. Analyze the Jet in the Truth Level

### 5-1. Preselection version 1

In [7]:
presel_bef, presel_pt, presel_pt_eta, presel_idx = myAnal_v3.preselection_v1(PseudoJet_filterDM_v3, pT_min=20, eta_max=2.5)

19373 events before preselection
19373 events after pT preselection
19373 events after pT & eta preselections
--------------------------------------------------------------------------------
0 events without PseudoJet before preselection
357 events without PseudoJet after pT preselection
519 events without PseudoJet after pT & eta preselections


### 5-2. Analyze truth jet with scheme 1 and version 1

In [8]:
N_jet_v1, jj_v1, jjj_v1 = myAnal_v3.analyze_truthJet_scheme1_v1(presel_pt_eta)

19373 events in the array of number of jets.
16752 selected events in dijet.
12771 selected events in trijet.


In [9]:
print(N_jet_v1.shape, jj_v1.shape, jjj_v1.shape)
print('-'*80)
print('dijet:')
print(jj_v1[0].shape, jj_v1[0])
print(jj_v1[9].shape, jj_v1[9])
print('-'*80)
print('trijet:')
print(jjj_v1[0].shape, jjj_v1[0])
print(jjj_v1[9].shape, jjj_v1[9])

(19373,) (10, 16752) (10, 12771)
--------------------------------------------------------------------------------
dijet:
(16752,) [144.81750682 156.63172343 163.66001185 ... 474.71395195  41.9887166
 562.2412089 ]
(16752,) [0.0000e+00 1.0000e+00 4.0000e+00 ... 1.9370e+04 1.9371e+04 1.9372e+04]
--------------------------------------------------------------------------------
trijet:
(12771,) [156.63172343 163.66001185 405.9696655  ... 474.71395195  41.9887166
 562.2412089 ]
(12771,) [1.0000e+00 4.0000e+00 5.0000e+00 ... 1.9370e+04 1.9371e+04 1.9372e+04]


### 5-3. Analyze truth jet with scheme 1 and version 2

In [10]:
N_jet_v2, jj_v2, jjj_v2 = myAnal_v3.analyze_truthJet_scheme1_v2(presel_pt_eta)

19373 events in the array of number of jets.
16752 selected events and 10 observables in dijet.
12771 selected events and 10 observables in trijet.


In [11]:
print(N_jet_v2.shape, jj_v2.shape, jjj_v2.shape)
print('-'*80)
print('dijet:')
print(jj_v2['M_jj'].head(3))
print(jj_v2['selected'].head(3))
print('-'*80)
print('trijet:')
print(jjj_v2['M_jjj'].head(3))
print(jjj_v2['selected'].head(3))

(19373, 1) (16752, 10) (12771, 10)
--------------------------------------------------------------------------------
dijet:
0     54.865697
1    277.092262
2    222.669041
Name: M_jj, dtype: float64
0    0.0
1    1.0
2    4.0
Name: selected, dtype: float64
--------------------------------------------------------------------------------
trijet:
0    445.684904
1    356.099158
2    858.190032
Name: M_jjj, dtype: float64
0    1.0
1    4.0
2    5.0
Name: selected, dtype: float64


In [13]:
N_jet_v2.head()

,N_jet
0,2
1,3
2,1
3,1
4,4


In [12]:
jj_v2.head()

,pT_1,pT_2,eta_1,eta_2,M_jj,MT_jj,mT_jj,Dphi,Deta,selected
0,144.817507,32.643869,0.389989,0.657986,54.865697,51.482040,182.523770,0.506671,0.267997,0.0
1,156.631723,116.276794,-0.720677,-1.840887,277.092262,225.132240,320.502725,1.909764,1.120210,1.0
2,163.660012,72.199409,-0.501624,1.195995,222.669041,68.053216,320.871248,0.440186,1.697619,4.0
3,405.969665,173.642818,0.164161,-0.849254,603.707680,533.288226,648.034045,2.995134,1.013415,5.0
4,73.993653,69.639708,-0.522255,-1.584679,86.456455,31.553025,165.587850,0.366903,1.062424,7.0


In [14]:
jjj_v2.head()

,pT_1,pT_2,pT_3,eta_1,eta_2,eta_3,M_jjj,MT_jjj,mT_jjj,selected
0,156.631723,116.276794,100.766976,-0.720677,-1.840887,-2.304753,445.684904,346.918388,470.524175,1.0
1,163.660012,72.199409,49.112062,-0.501624,1.195995,1.191665,356.099158,227.059823,399.885564,4.0
2,405.969665,173.642818,105.695284,0.164161,-0.849254,1.402649,858.190032,660.145111,881.950840,5.0
3,73.993653,69.639708,61.287772,-0.522255,-1.584679,0.814885,290.906597,170.318611,313.626423,7.0
4,349.992032,93.422654,53.809296,0.729602,-1.378293,0.022031,647.630734,431.982137,694.702131,8.0


### 5-4. Check `v1` and `v2` of scheme 1 functions

In [24]:
# check v1 and v2 are consistent
sum_N_jet = np.sum(np.transpose(N_jet_v2.to_numpy()) - N_jet_v1)
sum_jj = np.sum(np.transpose(jj_v2.to_numpy()) - jj_v1)
sum_jjj = np.sum(np.transpose(jjj_v2.to_numpy()) - jjj_v1)

print(sum_N_jet)
print(sum_jj)
print(sum_jjj)

0
0.0
0.0


##### Conclusion: The results of `v2` is the same as `v1` results.

## 6. Test

In [10]:
pt_1, pt_2 = [], []

for i in range(21,24):
    if np.shape(presel_pt_eta[i])[0] >= 2:
        test_pt, test_eta = presel_pt_eta[i]['pT'], presel_pt_eta[i]['eta']
        test_phi, test_m = presel_pt_eta[i]['phi'], presel_pt_eta[i]['mass']
        print("event {}".format(i))
        pt_1.append(test_pt[0])
        pt_2.append(test_pt[1])
        print("pT   =", test_pt)
        print("mass =", test_m)
    print('-'*80)
    
print("pT(1) = {}".format(pt_1))
print("pT(2) = {}".format(pt_2))

event 21
pT   = [754.72107953 104.83282762  85.25366785  54.1991487   27.54988231]
mass = [73.17201984 20.66261802 16.94153709 12.15247457  4.1833831 ]
--------------------------------------------------------------------------------
event 22
pT   = [135.81475624  23.67438139]
mass = [28.58375223  8.45196396]
--------------------------------------------------------------------------------
event 23
pT   = [492.57649468 415.89764971  42.02645083]
mass = [47.88720232 21.37220707 11.83759252]
--------------------------------------------------------------------------------
pT(1) = [754.7210795322543, 135.81475624404072, 492.57649467588806]
pT(2) = [104.83282762072056, 23.67438139019276, 415.8976497129115]


In [11]:
a = 3
if a == 2:
    print('yes 2')
else:
    print('no')
if a != 2:
    print('not 2')
if a == 3:
    print('yes 3')

no
not 2
yes 3


In [12]:
a = 3
if a >= 2:
    b = 4
    print('yes >= 2,', 'b =', b)
if a >= 3:
    b = 5
    print('yes >= 3,', 'b =', b)

yes >= 2, b = 4
yes >= 3, b = 5


In [18]:
test_observable = []
for i in range(5):
    print([i, i+1, 2*i])
    test_observable.append([i, i+1, 2*i])
arr_observable = np.array(test_observable)
print('-'*80)
print(test_observable)
print("{} events and {} observables.".format(arr_observable.shape[0],
                                             arr_observable.shape[1]))
arr_observable

[0, 1, 0]
[1, 2, 2]
[2, 3, 4]
[3, 4, 6]
[4, 5, 8]
--------------------------------------------------------------------------------
[[0, 1, 0], [1, 2, 2], [2, 3, 4], [3, 4, 6], [4, 5, 8]]
5 events and 3 observables.


array([[0, 1, 0],
       [1, 2, 2],
       [2, 3, 4],
       [3, 4, 6],
       [4, 5, 8]])